In [107]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture

# Récuperation du json et conversion en quelque chose d'utilisable

In [108]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [109]:
contenu_fichier


'[{"modele":"208","carburant":"Essence","prix":17990,"kilometrage":8556,"garantie_kilometrage":"n","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Blanc","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2022,"puissance":75,"silhouette":"Citadine","nb_places":5,"utilisation_prec":"Ex-Loueur","puissance_fiscale":4,"critair":"1","ptac":1510,"nb_portes":5},{"modele":"RIFTER","carburant":"Diesel","prix":27490,"kilometrage":51739,"garantie_kilometrage":"g","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Brun","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2020,"puissance":131,"silhouette":"Familiale","nb_places":5,"utilisation_prec":"Ex-Particulier","puissance_fiscale":7,"critair":"2","ptac":2135,"nb_portes":5},{"modele":"208","carburant":"Essence","prix":18970,"kilometrage":40071,"garantie_kilometrage":"n","boite_de_vitesse":"Manuelle","transmission":"2","couleur":"Gris","garantie":"SPOTICAR PREMIUM","date_mise_circulation":2021,"puissance":102,"silhouett

In [110]:
reconstitution = from_json(list[Voiture],contenu_fichier)



In [111]:
reconstitution[0]

Voiture(modele='208', carburant='Essence', prix=17990, kilometrage=8556, garantie_kilometrage='n', boite_de_vitesse='Manuelle', transmission='2', couleur='Blanc', garantie='SPOTICAR PREMIUM', date_mise_circulation=2022, puissance=75, silhouette='Citadine', nb_places=5, utilisation_prec='Ex-Loueur', puissance_fiscale=4, critair='1', ptac=1510, nb_portes=5)

# Conversion en base de données panda et nettoyage

In [112]:
import pandas as pd

In [113]:
df = pd.DataFrame(reconstitution)

In [114]:
df.describe()

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,ptac
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,27001.200000,28036.790000,2020.880000,123.630000,5.070000,6.290000,1843.260000
std,7129.965968,24968.844017,1.112645,21.570956,0.670519,1.437555,253.874487
min,14990.000000,5.000000,2018.000000,75.000000,2.000000,4.000000,1510.000000
25%,21620.000000,9407.000000,2020.750000,102.000000,5.000000,5.000000,1641.250000
50%,26370.000000,21772.000000,2021.000000,131.000000,5.000000,6.500000,1770.000000
75%,30235.000000,44125.000000,2022.000000,131.000000,5.000000,7.000000,1950.000000
max,48970.000000,99000.000000,2022.000000,181.000000,9.000000,10.000000,3030.000000


In [115]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,208,Essence,17990,8556,n,Manuelle,2,Blanc,SPOTICAR PREMIUM,2022,75,Citadine,5,Ex-Loueur,4,1,1510,5
1,RIFTER,Diesel,27490,51739,g,Manuelle,2,Brun,SPOTICAR PREMIUM,2020,131,Familiale,5,Ex-Particulier,7,2,2135,5
2,208,Essence,18970,40071,n,Manuelle,2,Gris,SPOTICAR PREMIUM,2021,102,Citadine,5,Ex-Loueur,5,1,1595,5
3,2008,Essence,26970,15512,n,Automatique,2,Gris,SPOTICAR PREMIUM,2021,131,SUV-4x4,5,Ex-Loueur,7,1,1752,5
4,3008,Diesel,22970,73499,n,Manuelle,2,Rouge,SPOTICAR PREMIUM,2019,130,SUV-4x4,5,Ex-Loueur,6,2,1970,5


In [116]:
df.dtypes

modele                   object
carburant                object
prix                      int64
kilometrage               int64
garantie_kilometrage     object
boite_de_vitesse         object
transmission             object
couleur                  object
garantie                 object
date_mise_circulation     int64
puissance                 int64
silhouette               object
nb_places                 int64
utilisation_prec         object
puissance_fiscale         int64
critair                  object
ptac                      int64
nb_portes                object
dtype: object

In [117]:
df.modele.unique() #supprimer les espaces ?

array(['208', 'RIFTER', '2008', '3008', '308', '5008', '508 SW', 'EXPERT',
       '308 SW', 'PARTNER', 'TRAVELLER'], dtype=object)

In [118]:
df.carburant.unique()#supprimer les espaces ?

array(['Essence', 'Diesel', 'Hybride rechargeable', 'Electrique'],
      dtype=object)

In [119]:
for i in df.columns:
    col_selec = f"df.{i}.unique()"
    
    print(f"{i} : {eval(col_selec)} ")

modele : ['208' 'RIFTER' '2008' '3008' '308' '5008' '508 SW' 'EXPERT' '308 SW'
 'PARTNER' 'TRAVELLER'] 
carburant : ['Essence' 'Diesel' 'Hybride rechargeable' 'Electrique'] 
prix : [17990 27490 18970 26970 22970 21470 23770 20470 17970 22770 24470 22470
 30970 27470 24970 27970 28470 21670 19970 43970 45470 29970 38470 29470
 20970 20770 17470 28970 31470 31970 37970 36770 21970 27770 38770 28770
 16970 15770 15470 33470 25970 19470 23470 26770 31770 33770 32970 39970
 37470 35970 18770 19770 44970 34970 18470 48970 27990 14990 29990 23990] 
kilometrage : [ 8556 51739 40071 15512 73499 25086 24416 30583 49727 19165  8126 16818
 21919 45800 16793 19281 99000 22422 33219 59795    10  8436 26370 31037
 19388 10212 31335 22280  3000 21625 15713 10000 23199 51640 25000 75000
  9461 44500 33000 77125 48997 20743  2762 38840 97034 75249 18267 18340
 51306 89000  9245 14461  4486 22091  9990 69728 48662 31500 20975  7111
  2473 19000     5  5436 24139 45159 21272 44000 32538 10013 15452 22001


## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

In [120]:
df.dtypes

modele                   object
carburant                object
prix                      int64
kilometrage               int64
garantie_kilometrage     object
boite_de_vitesse         object
transmission             object
couleur                  object
garantie                 object
date_mise_circulation     int64
puissance                 int64
silhouette               object
nb_places                 int64
utilisation_prec         object
puissance_fiscale         int64
critair                  object
ptac                      int64
nb_portes                object
dtype: object

In [121]:
df.convert_dtypes().dtypes

modele                   string
carburant                string
prix                      Int64
kilometrage               Int64
garantie_kilometrage     string
boite_de_vitesse         string
transmission             string
couleur                  string
garantie                 string
date_mise_circulation     Int64
puissance                 Int64
silhouette               string
nb_places                 Int64
utilisation_prec         string
puissance_fiscale         Int64
critair                  string
ptac                      Int64
nb_portes                object
dtype: object

In [122]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,208,Essence,17990,8556,n,Manuelle,2,Blanc,SPOTICAR PREMIUM,2022,75,Citadine,5,Ex-Loueur,4,1,1510,5
1,RIFTER,Diesel,27490,51739,g,Manuelle,2,Brun,SPOTICAR PREMIUM,2020,131,Familiale,5,Ex-Particulier,7,2,2135,5
2,208,Essence,18970,40071,n,Manuelle,2,Gris,SPOTICAR PREMIUM,2021,102,Citadine,5,Ex-Loueur,5,1,1595,5
3,2008,Essence,26970,15512,n,Automatique,2,Gris,SPOTICAR PREMIUM,2021,131,SUV-4x4,5,Ex-Loueur,7,1,1752,5
4,3008,Diesel,22970,73499,n,Manuelle,2,Rouge,SPOTICAR PREMIUM,2019,130,SUV-4x4,5,Ex-Loueur,6,2,1970,5


In [164]:
#df.garantie_kilometrage= df.garantie_kilometrage.astype('bool')
#df.modele= df.garantie_kilometrage.astype('category')
df = df.astype({ "modele": 'object',"carburant": 'category',
                "prix": 'int32',
                "kilometrage" : 'int32',
#                "garantie_kilometrage":'bool',
#                "boite_de_vitesse": 'bool', #checker sur plus grosses données si il existe d'autres modalités
                "transmission" : 'category',
                "couleur" : 'category',
                "garantie" : 'category',
                "date_mise_circulation":'category',
                "puissance" : "int32",
                "silhouette" : 'category',
                "nb_places" : 'int32',
                "utilisation_prec" : 'category',
                "puissance_fiscale" : 'int32',
                "critair": 'category',
                "boite_de_vitesse": 'category',
                "ptac" : "int32"
                })

In [165]:
df.dtypes

modele                     object
carburant                category
prix                        int32
kilometrage                 int32
garantie_kilometrage         bool
boite_de_vitesse         category
transmission             category
couleur                  category
garantie                 category
date_mise_circulation    category
puissance                   int32
silhouette               category
nb_places                   int32
utilisation_prec         category
puissance_fiscale           int32
critair                  category
ptac                        int32
nb_portes                  object
dtype: object

In [166]:
df.nb_portes.str.match('NA').value_counts()

True    1
Name: nb_portes, dtype: int64

In [167]:
#df.garantie_kilometrage.replace({'n': 'no', 'g': 'yes'})

In [168]:
df.garantie_kilometrage.unique()

array([False,  True])

In [169]:
df['garantie_kilometrage'] = df['garantie_kilometrage'].map({'n':False,'g':True})


In [179]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

       modele             carburant   prix  kilometrage garantie_kilometrage  \
0         208               Essence  17990         8556                  NaN   
1      RIFTER                Diesel  27490        51739                  NaN   
2         208               Essence  18970        40071                  NaN   
3        2008               Essence  26970        15512                  NaN   
4        3008                Diesel  22970        73499                  NaN   
5         208               Essence  21470        25086                  NaN   
6         208               Essence  23770        24416                  NaN   
7         208                Diesel  20470        30583                  NaN   
8         208               Essence  17970        49727                  NaN   
9         208               Essence  22770        19165                  NaN   
10        208               Essence  24470         8126                  NaN   
11        208               Essence  224

In [171]:
df.dtypes

modele                     object
carburant                category
prix                        int32
kilometrage                 int32
garantie_kilometrage       object
boite_de_vitesse         category
transmission             category
couleur                  category
garantie                 category
date_mise_circulation    category
puissance                   int32
silhouette               category
nb_places                   int32
utilisation_prec         category
puissance_fiscale           int32
critair                  category
ptac                        int32
nb_portes                  object
dtype: object

In [172]:
df.modele.unique()

array(['208', 'RIFTER', '2008', '3008', '308', '5008', '508 SW', 'EXPERT',
       '308 SW', 'PARTNER', 'TRAVELLER'], dtype=object)

In [173]:
y = df['prix'].to_numpy()
X = df.loc[:, df.columns != 'prix'].to_numpy() # tout sauf le prix

In [174]:
type(y), y.shape

(numpy.ndarray, (100,))

In [175]:
type(X), X.shape

(numpy.ndarray, (100, 17))

In [176]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet

In [177]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [178]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr)
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

ValueError: 
All the 300 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py", line 911, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 554, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bapti\Ap

In [163]:
print(X)

[['208' 'Essence' 8556 ... '1' 1510 5]
 ['RIFTER' 'Diesel' 51739 ... '2' 2135 5]
 ['208' 'Essence' 40071 ... '1' 1595 5]
 ...
 ['208' 'Diesel' 89636 ... '2' 1645 5]
 ['5008' 'Essence' 51474 ... '1' 2150 5]
 ['3008' 'Essence' 75611 ... '1' 1930 5]]


In [180]:
print(X_tr)

[['308' 'Hybride rechargeable' 28000 ... '1' 2120 5]
 ['308 SW' 'Diesel' 2473 ... '2' 1920 5]
 ['3008' 'Diesel' 73499 ... '2' 1970 5]
 ...
 ['208' 'Essence' 8436 ... '1' 1625 5]
 ['3008' 'Essence' 33000 ... '1' 1940 5]
 ['2008' 'Essence' 22001 ... '1' 1752 5]]
